In [92]:
import pandas as pd
import numpy as np
import string
import nltk
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns

In [93]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [94]:
!kaggle dataset files -d dbdmobile/myanimelist-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [95]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dbdmobile/myanimelist-dataset",path="anime-dataset-2023.csv")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/myanimelist-dataset/anime-dataset-2023.csv


In [96]:
anime=pd.read_csv('/kaggle/input/myanimelist-dataset/anime-dataset-2023.csv')

In [97]:
anime.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


In [98]:
anime.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

In [99]:
anime[anime['Producers']!='UNKNOWN']['Producers'].sample(1)

,Producers
360,Fuji TV


In [100]:
anime=anime.drop(['Duration','Status','Image URL','anime_id','Other name','Licensors','Members','Premiered','Scored By'],axis=1)

In [101]:
anime.head()
anime=anime.rename(columns={'title_english':'title'})

In [102]:
anime.columns

Index(['Name', 'English name', 'Score', 'Genres', 'Synopsis', 'Type',
       'Episodes', 'Aired', 'Producers', 'Studios', 'Source', 'Rating', 'Rank',
       'Popularity', 'Favorites'],
      dtype='object')

In [103]:
(anime.isna().sum())

,0
Name,0
English name,0
Score,0
Genres,0
Synopsis,0
Type,0
Episodes,0
Aired,0
Producers,0
Studios,0


Only column 'Popularity' and 'Favorites' are Integers rest are String

In [104]:
anime.dtypes

,0
Name,object
English name,object
Score,object
Genres,object
Synopsis,object
Type,object
Episodes,object
Aired,object
Producers,object
Studios,object


In [105]:
anime.head()

,Name,English name,Score,Genres,Synopsis,Type,Episodes,Aired,Producers,Studios,Source,Rating,Rank,Popularity,Favorites
0,Cowboy Bebop,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",Bandai Visual,Sunrise,Original,R - 17+ (violence & profanity),41.0,43,78525
1,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001","Sunrise, Bandai Visual",Bones,Original,R - 17+ (violence & profanity),189.0,602,1448
2,Trigun,Trigun,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",Victor Entertainment,Madhouse,Manga,PG-13 - Teens 13 or older,328.0,246,15035
3,Witch Hunter Robin,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002","Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,Original,PG-13 - Teens 13 or older,2764.0,1795,613
4,Bouken Ou Beet,Beet the Vandel Buster,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005","TV Tokyo, Dentsu",Toei Animation,Manga,PG - Children,4240.0,5126,14


In [106]:
anime.duplicated().sum()

np.int64(0)

In [107]:
def convert_genres(str):
   str=str.replace('[','')
   str=str.replace(']','')
   str=str.replace("'",'')
   str=str.replace(' ','')
   str=str.lower()
   res=str.split(',')
   return res

In [108]:
convert_genres(anime.iloc[0].Genres)

['action', 'awardwinning', 'sci-fi']

In [109]:
anime['Genres']=anime['Genres'].apply(convert_genres)

In [110]:
anime['Synopsis']=anime['Synopsis'].apply(lambda x:x.split())

In [111]:
def num_2_string(obj):
  if type(obj)==int or type(obj)==float:
    return [str(obj)]
  else:
    return obj

In [112]:
anime['Popularity']=anime['Popularity'].apply(num_2_string)
anime['Favorites']=anime['Favorites'].apply(num_2_string)

In [113]:
anime.columns

Index(['Name', 'English name', 'Score', 'Genres', 'Synopsis', 'Type',
       'Episodes', 'Aired', 'Producers', 'Studios', 'Source', 'Rating', 'Rank',
       'Popularity', 'Favorites'],
      dtype='object')

In [114]:
anime.dtypes

,0
Name,object
English name,object
Score,object
Genres,object
Synopsis,object
Type,object
Episodes,object
Aired,object
Producers,object
Studios,object


In [115]:
anime['Popularity']=anime['Popularity'].apply(num_2_string)
anime['Favorites']=anime['Favorites'].apply(num_2_string)
for col in anime.columns:
  if col not in ['Popularity','Favorites','Synopsis','Genres']:
   anime[col]=anime[col].apply(lambda x:[x])

In [116]:
anime.head()

,Name,English name,Score,Genres,Synopsis,Type,Episodes,Aired,Producers,Studios,Source,Rating,Rank,Popularity,Favorites
0,[Cowboy Bebop],[Cowboy Bebop],[8.75],"[action, awardwinning, sci-fi]","[Crime, is, timeless., By, the, year, 2071,, h...",[TV],[26.0],"[Apr 3, 1998 to Apr 24, 1999]",[Bandai Visual],[Sunrise],[Original],[R - 17+ (violence & profanity)],[41.0],[43],[78525]
1,[Cowboy Bebop: Tengoku no Tobira],[Cowboy Bebop: The Movie],[8.38],"[action, sci-fi]","[Another, day,, another, bounty—such, is, the,...",[Movie],[1.0],"[Sep 1, 2001]","[Sunrise, Bandai Visual]",[Bones],[Original],[R - 17+ (violence & profanity)],[189.0],[602],[1448]
2,[Trigun],[Trigun],[8.22],"[action, adventure, sci-fi]","[Vash, the, Stampede, is, the, man, with, a, $...",[TV],[26.0],"[Apr 1, 1998 to Sep 30, 1998]",[Victor Entertainment],[Madhouse],[Manga],[PG-13 - Teens 13 or older],[328.0],[246],[15035]
3,[Witch Hunter Robin],[Witch Hunter Robin],[7.25],"[action, drama, mystery, supernatural]","[Robin, Sena, is, a, powerful, craft, user, dr...",[TV],[26.0],"[Jul 3, 2002 to Dec 25, 2002]","[Bandai Visual, Dentsu, Victor Entertainment, ...",[Sunrise],[Original],[PG-13 - Teens 13 or older],[2764.0],[1795],[613]
4,[Bouken Ou Beet],[Beet the Vandel Buster],[6.94],"[adventure, fantasy, supernatural]","[It, is, the, dark, century, and, the, people,...",[TV],[52.0],"[Sep 30, 2004 to Sep 29, 2005]","[TV Tokyo, Dentsu]",[Toei Animation],[Manga],[PG - Children],[4240.0],[5126],[14]


In [117]:
type(anime.iloc[0]['Popularity'][0])

str

In [118]:
anime.columns

Index(['Name', 'English name', 'Score', 'Genres', 'Synopsis', 'Type',
       'Episodes', 'Aired', 'Producers', 'Studios', 'Source', 'Rating', 'Rank',
       'Popularity', 'Favorites'],
      dtype='object')

In [119]:
anime['tags']=anime['Genres']+anime['Synopsis']+anime['Popularity']+anime['Favorites']+anime['Type']+anime['Source']+anime['Studios']+anime['Producers']+anime['Rating']+anime['Rank']+anime['Episodes']

In [120]:
anime=anime[['Name','tags']]

In [121]:
anime

,Name,tags
0,[Cowboy Bebop],"[action, awardwinning, sci-fi, Crime, is, time..."
1,[Cowboy Bebop: Tengoku no Tobira],"[action, sci-fi, Another, day,, another, bount..."
2,[Trigun],"[action, adventure, sci-fi, Vash, the, Stamped..."
3,[Witch Hunter Robin],"[action, drama, mystery, supernatural, Robin, ..."
4,[Bouken Ou Beet],"[adventure, fantasy, supernatural, It, is, the..."
...,...,...
24900,[Wu Nao Monu],"[comedy, fantasy, sliceoflife, No, description..."
24901,[Bu Xing Si: Yuan Qi],"[action, adventure, fantasy, No, description, ..."
24902,[Di Yi Xulie],"[action, adventure, fantasy, sci-fi, No, descr..."
24903,[Bokura no Saishuu Sensou],"[unknown, A, music, video, for, the, song, Bok..."


In [122]:
anime['tags'].isna().sum()

np.int64(0)

In [123]:
anime.shape

(24905, 2)

In [124]:
anime['tags'].sample(1)

,tags
8981,"[action, adventure, fantasy, In, a, world, sim..."


In [125]:
stopwords=set(stopwords.words('english'))
def remove_stopwords(obj):
  res=[word.lower() for word in obj if word not in stopwords]
  return res
def lemmatize(obj):
  lemmatizer=WordNetLemmatizer()
  res=[lemmatizer.lemmatize(word) for word in obj]
  return res


In [126]:
anime['tags']=anime['tags'].apply(remove_stopwords)
anime['tags']=anime['tags'].apply(lemmatize)

In [127]:
anime

,Name,tags
0,[Cowboy Bebop],"[action, awardwinning, sci-fi, crime, timeless..."
1,[Cowboy Bebop: Tengoku no Tobira],"[action, sci-fi, another, day,, another, bount..."
2,[Trigun],"[action, adventure, sci-fi, vash, stampede, ma..."
3,[Witch Hunter Robin],"[action, drama, mystery, supernatural, robin, ..."
4,[Bouken Ou Beet],"[adventure, fantasy, supernatural, it, dark, c..."
...,...,...
24900,[Wu Nao Monu],"[comedy, fantasy, sliceoflife, no, description..."
24901,[Bu Xing Si: Yuan Qi],"[action, adventure, fantasy, no, description, ..."
24902,[Di Yi Xulie],"[action, adventure, fantasy, sci-fi, no, descr..."
24903,[Bokura no Saishuu Sensou],"[unknown, a, music, video, song, bokura, saish..."


In [128]:
anime.iloc[2,1]

['action',
 'adventure',
 'sci-fi',
 'vash',
 'stampede',
 'man',
 '$$60,000,000,000',
 'bounty',
 'head.',
 'the',
 'reason:',
 'merciless',
 'villain',
 'lay',
 'waste',
 'oppose',
 'flattens',
 'entire',
 'city',
 'fun,',
 'garnering',
 'title',
 '"the',
 'humanoid',
 'typhoon."',
 'he',
 'leaf',
 'trail',
 'death',
 'destruction',
 'wherever',
 'goes,',
 'anyone',
 'count',
 'dead',
 'much',
 'make',
 'eye',
 'contact—or',
 'rumor',
 'say.',
 'in',
 'actuality,',
 'vash',
 'huge',
 'softie',
 'claim',
 'never',
 'taken',
 'life',
 'avoids',
 'violence',
 'costs.',
 'with',
 'crazy',
 'doughnut',
 'obsession',
 'buffoonish',
 'attitude',
 'tow,',
 'vash',
 'traverse',
 'wasteland',
 'planet',
 'gunsmoke,',
 'followed',
 'two',
 'insurance',
 'agents,',
 'meryl',
 'stryfe',
 'milly',
 'thompson,',
 'attempt',
 'minimize',
 'impact',
 'public.',
 'but',
 'soon,',
 'misadventure',
 'evolve',
 'life-or-death',
 'situation',
 'group',
 'legendary',
 'assassin',
 'summoned',
 'bring',
 's

In [129]:
def list2string(obj):
  sent=""
  if(isinstance(obj,list)):
    for a in obj:
      sent=sent+a+" "
  return sent

In [130]:
anime['tags']=anime['tags'].apply(list2string)
anime['Name']=anime['Name'].apply(lambda x:x[0])

In [131]:
anime

,Name,tags
0,Cowboy Bebop,action awardwinning sci-fi crime timeless. by ...
1,Cowboy Bebop: Tengoku no Tobira,"action sci-fi another day, another bounty—such..."
2,Trigun,action adventure sci-fi vash stampede man $$60...
3,Witch Hunter Robin,action drama mystery supernatural robin sena p...
4,Bouken Ou Beet,adventure fantasy supernatural it dark century...
...,...,...
24900,Wu Nao Monu,comedy fantasy sliceoflife no description avai...
24901,Bu Xing Si: Yuan Qi,action adventure fantasy no description availa...
24902,Di Yi Xulie,action adventure fantasy sci-fi no description...
24903,Bokura no Saishuu Sensou,unknown a music video song bokura saishuu sens...


In [132]:
cv=CountVectorizer(max_features=500)

In [133]:
vectors=cv.fit_transform(anime['tags']).toarray()

In [134]:
vectors.shape

(24905, 500)

In [135]:
similarity=cosine_similarity(vectors)


In [45]:
similarity

array([[0.56818182, 0.14301939, 0.17815671, ..., 0.        , 0.        ,
        0.        ],
       [0.14301939, 0.50249857, 0.08304548, ..., 0.        , 0.        ,
        0.        ],
       [0.17815671, 0.08304548, 0.53448276, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.57154761,
        0.57154761],
       [0.        , 0.        , 0.        , ..., 0.57154761, 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.57154761, 1.        ,
        1.        ]])

In [138]:
anime[anime['Name']=='Jujutsu Kaisen'].index

Index([16236], dtype='int64')

In [141]:
def recommend(name):
  idx=anime[anime['Name']==name].index[0]
  distances=similarity[idx]
  anime_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:20]
  for i in anime_list:
    print(anime.iloc[i[0]].Name)

In [142]:
recommend(r"Jujutsu Kaisen")

Ajin
Jujutsu Kaisen 0 Movie
Bleach: Sennen Kessen-hen
Jujutsu Kaisen 2nd Season
Ao no Exorcist
Akame ga Kill! Recap
Noblesse: Awakening
Osomatsu-san Movie
Tokyo Ghoul
Kore wa Zombie desu ka?
Sword Gai The Animation Part II
Fuuma no Kojirou: Seiken Sensou-hen
Tokyo Ghoul: "Pinto"
Ajin Part 2
Cyborg 009 VS Devilman
Kimetsu no Yaiba: Yuukaku-hen
Tomodachi Game
Fuuma no Kojirou: Fuuma Hanran-hen
Kekkai Sensen: Soresaemo Saitei de Saikou na Hibi
